In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# --- Définitions de fonctions et paramètres de base ---
# (Ces fonctions sont supposées être définies dans votre environnement)

# Exemple de définition de W (largeur du puits)
W = 18

def energy(n, a, hbar=1, m=1):
    return (n**2 * np.pi**2 * hbar**2) / (2 * m * a**2)

def psi_n_whole(x, Nmax, W):
    psi_vals = []
    for n in range(1, Nmax+1):
        psi_n = np.where((x >= 0) & (x <= W), np.sqrt(2/W)*np.sin(n * np.pi * x/W), 0)
        psi_vals.append(psi_n)
    return np.array(psi_vals)

def check_normalization(psi_func, a):
    psi_vals = psi_func(x, a)
    norm = np.trapz(np.abs(psi_vals)**2, x)
    return norm

def psi_init(x, a, A=1):
    return A * np.exp(-(x - a) ** 2)

Nmax = 20  # Nombre maximal de niveaux quantiques
time_steps = 1000  # Nombre de pas de temps pour l'animation
total_time = 3*69  # Temps total de l'animation
dt = total_time / time_steps  # Durée d'un pas de temps

# Définition de la grille spatiale
x = np.linspace(-5, W+5, 1000)

# Pré-calcul des fonctions propres pour tous les n
psi_n_values = np.array(psi_n_whole(x, Nmax, W))

# Calcul des coefficients phi_n pour un état initial psi_0(x) donné
def phi_n(a):
    psi_values = psi_init(x, a)
    area = check_normalization(psi_init, a)  # Intégration sur [-∞, ∞]
    psi_init_normalized = psi_init(x, a, A=1/np.sqrt(area))
    phi_x_a = psi_init_normalized
    func = phi_x_a * psi_n_values
    coeffs = np.trapz(func, x)
    return coeffs

# Pré-calcule phi_n pour différentes valeurs de a
w_values = [W/2, W/3, W/6, W/4]
w_strs = ["$W/2$", "$W/3$", "$W/6$", "$W/4$"]

# Pour la boucle dans animate, on définit a_values égale à w_values
a_values = w_values

phi_n_coeffs_arr = np.array([phi_n(a) for a in w_values])

# Vecteur temps
exponentials_energies = np.exp(
    -1j * np.outer(energy(np.arange(1, Nmax + 1), W), np.linspace(0, total_time, time_steps))
)

# Pré-calcul de la fonction d'onde dépendante du temps Φ(x, t)
phi_x_t = np.array(
    [
        (
            psi_n_values[:, :, np.newaxis]
            * exponentials_energies[:, np.newaxis, :]
            * phi_n_coeffs[:, np.newaxis, np.newaxis]
        ).sum(axis=0)
        for phi_n_coeffs in phi_n_coeffs_arr
    ]
)

# Création et configuration des sous-graphes
fig, axes = plt.subplots(2, 2, figsize=(15, 8))
plt.tight_layout(pad=5.0)
axes = axes.flatten()
for i, ax in enumerate(axes):
    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(0, np.max(np.abs(phi_x_t[i]) ** 2))
    ax.set_ylabel("Probability Density $|\Phi(x, t)|^2$")
    ax.set_xlabel("Position x")

# =================== AJOUT : Tracé de la fonction d'onde initiale ===================
# Pour chaque valeur de a (issue de w_values), on calcule et trace la densité de probabilité
# de l'état initial psi_init (normalisé) en pointillés.
for idx, a in enumerate(w_values):
    area = check_normalization(psi_init, a)
    psi_init_normalized = psi_init(x, a, A=1/np.sqrt(area))
    axes[idx].plot(x, np.abs(psi_init_normalized)**2, 'k--', lw=1.5, label="Initial $|\psi(x)|^2$")
    axes[idx].legend()
# ================================================================================

line = [axes[i].plot([], [], lw=2)[0] for i in range(4)]
for idx in range(4):
    axes[idx].set_title(r"Evolution for {}".format(w_strs[idx]))

# ---- AJOUT pour indiquer le temps caractéristique ----
# Calcul du temps caractéristique t_c = π*hbar/(E2 - E1) (ici hbar=1)
tc = np.pi / (energy(2, W) - energy(1, W))
# Calcul des indices de frame correspondants :
frame_tc     = int(tc / dt)         # pour t = t_c
frame_3over4 = int((3/4 * tc) / dt)   # pour t = 3/4 t_c
frame_3tc    = int((3 * tc) / dt)     # pour t = 3 t_c

print("tc =", tc, "frame_tc =", frame_tc, "frame_3/4tc =", frame_3over4, "frame_3tc =", frame_3tc)

# Création d'une annotation texte sur chaque sous-graphe
text_annotations = [axes[i].text(0.05, 0.9, "", transform=axes[i].transAxes, color="red", fontsize=12)
                    for i in range(4)]
# -------------------------------------------------------

# Fonction d'animation (modifiée pour afficher une indication permanente une fois que le temps requis est atteint)
def animate(i):
    for idx, (a, label) in enumerate(zip(a_values, w_strs)):
        line[idx].set_data(x, np.abs(phi_x_t[idx, :, i]) ** 2)
        # Conditions spécifiques selon l'indice (correspondant aux positions initiales)
        if idx == 0:  # Pour la gaussienne centrée en W/2 : refocalisation à t = 3/4 t_c
            if i >= frame_3over4:
                text_annotations[idx].set_text("t = 3/4 t_c reached")
            else:
                text_annotations[idx].set_text("")
        elif idx == 1:  # Pour la gaussienne centrée en W/3 : refocalisation à t = t_c
            if i >= frame_tc:
                text_annotations[idx].set_text("t = t_c reached")
            else:
                text_annotations[idx].set_text("")
        elif idx == 2:  # Pour la gaussienne centrée en W/6 : refocalisation à t = 3 t_c
            if i >= frame_3tc:
                text_annotations[idx].set_text("t = 3 t_c reached")
            else:
                text_annotations[idx].set_text("")
        else:
            # Pour la gaussienne en W/4, aucune indication n'est spécifiée
            text_annotations[idx].set_text("")
    return line + text_annotations

# Animation avec interval=50 (à ajuster si nécessaire)
ani = FuncAnimation(fig, animate, frames=time_steps, interval=50, blit=True)

# Sauvegarde de l'animation en MP4
ani.save("time_infinite_well_gaussian_evolution.mp4", writer="ffmpeg", fps=30)

plt.show()


/var/folders/0r/nnn8yqwd549bhsr3jqz9kqyh0000gn/T/ipykernel_61864/3085536284.py:23: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  norm = np.trapz(np.abs(psi_vals)**2, x)
/var/folders/0r/nnn8yqwd549bhsr3jqz9kqyh0000gn/T/ipykernel_61864/3085536284.py:47: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  coeffs = np.trapz(func, x)


tc = 68.75493541569878 frame_tc = 332 frame_3/4tc = 249 frame_3tc = 996


In [ ]:
from scipy.integrate import quad
def psi_init(x, a, A=1):
    return A * np.exp(-(x - a) ** 2)

def check_norm(psi_func, a, A=1):
    """
    Vérifie l'aire sous la courbe de la densité de probabilité |psi(x)|^2 en intégrant sur [-inf, inf].
    """
    prob_density = lambda x: np.abs(psi_func(x, a, A))**2  # Fonction à intégrer
    area, _ = quad(prob_density, -np.inf, np.inf)  # Intégration sur [-inf, inf]
    return area

# Définition des paramètres
dx = 0.01
x0 = 0
W = 18
dt = 0.01
hbar = 1
m = 1
t = 0

# Création des valeurs de x
x = np.arange(0, W, dx)

# Évaluation de psi_0(x)
psi_init_values = psi_init(x, W/2)

# Vérification de la normalisation
area = check_norm(psi_init, a)  # Intégration sur [-inf, inf]

# Normalisation de psi_0(x)
psi_init_normalized = psi_init(x, W/2, A=1/np.sqrt(area))

# Création de la figure avec deux sous-graphiques
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Premier graphique : Fonction d'onde psi_0(x)
axes[0].plot(x, psi_init_normalized, label=r"$\phi(x,0)$", color="red")
axes[0].set_xlabel("x")
axes[0].set_ylabel(r"$\phi(x,0)$")
axes[0].set_title("Fonction d'onde $\phi(x,0)$")
axes[0].legend()
axes[0].grid()

# Deuxième graphique : Densité de probabilité |ψ_0(x)|^2
axes[1].plot(x, np.abs(psi_init_normalized)**2, label=r"$|\phi(x,0)|^2$", color="blue")
axes[1].set_xlabel("x")
axes[1].set_ylabel(r"$|\phi(x,0)|^2$")
axes[1].set_title("Densité de probabilité $|\phi(x,0)|^2$")
axes[1].legend()
axes[1].grid()

# Affichage des graphiques
plt.tight_layout()
plt.show()

area3 = check_norm(psi_init, W/2,  1/np.sqrt(area))  # Intégration sur [-inf, inf]
print(area3)

In [ ]:
# Paramètres du puits de potentiel carré infini
a = 1.        # largeur du puits
hbar = 1.0
m = 1.0

# Fonction énergie pour le puits infini
def energy(n, a, hbar=1, m=1):
    """
    Calcule l'énergie du niveau quantique n pour une particule dans une boîte 1D de largeur a.
    """
    return (n**2 * np.pi**2 * hbar**2) / (2 * m * a**2)

# Définition des fonctions propres pour le puits infini
def psi(n, x, a):
    """
    Calcule la fonction propre psi_n pour le puits infini.
    La fonction est non nulle uniquement dans l'intervalle [0,a].
    """
    # Utilisation de np.where pour définir psi(x)=sqrt(2/a)*sin(n*pi*x/a) pour 0<=x<=a, et 0 sinon.
    return np.where((x >= 0) & (x <= a), 4*np.sqrt(2/a) * np.sin(n * np.pi * x / a), 0)

# Définition de la gamme en x
# Pour mieux visualiser, on définit x sur un intervalle élargi autour du puits.
x_values = np.linspace(-0.2 * a, 1.2 * a, 1000)

# Définition du potentiel du puits infini
# On attribue une valeur très élevée (V0) en dehors du puits afin de représenter une barrière « infinie »
V0 = 150  # valeur haute pour représenter l'infini en dehors de [0,a]
def infinite_square_well_potential(x, a, V0=150):
    """
    Retourne le potentiel V(x) qui vaut 0 pour 0 <= x <= a et V0 ailleurs.
    """
    return np.where((x >= 0) & (x <= a), 0, V0)

potential = infinite_square_well_potential(x_values, a, V0)

# Calcul des énergies et fonctions propres pour les 4 premiers niveaux (n = 1, 2, 3, 4)
niveaux = [1, 2, 3, 4]
energies = [energy(n, a, hbar, m) for n in niveaux]
psi_vals = [psi(n, x_values, a) for n in niveaux]
# Densités de probabilité
prob_vals = [psi_val**2 for psi_val in psi_vals]

# Couleurs pour différencier les niveaux
colors = ["blue", "orange", "green", "red"]

# Initialisation des sous-graphes
fig, ax = plt.subplots(1, 2, figsize=(18, 6), sharey=True)

# --- Graphique des fonctions d'onde décalées par leur niveau d'énergie ---
ax[0].plot(x_values, potential, label="Puits de potentiel\n(côtés élevés)", color='black', linestyle="-")

for n, (psi_val, E, color) in zip(niveaux, zip(psi_vals, energies, colors)):
    # Décalage vertical par le niveau d'énergie
    psi_shifted = psi_val + E
    ax[0].plot(x_values, psi_shifted, label=r"$\Psi_{{{}}}(x)$".format(n), color=color)
    # Tracer des lignes verticales aux bords du puits (points de "rebond")
    ax[0].vlines(0, ymin=0, ymax=E, linestyles='dashed', colors=color)
    ax[0].vlines(a, ymin=0, ymax=E, linestyles='dashed', colors=color)
    # Remplissage entre le niveau d'énergie et la fonction d'onde (uniquement dans [0,a])
    ax[0].fill_between(x_values, E, psi_shifted, where=((x_values >= 0) & (x_values <= a)), 
                        color=color, alpha=0.3)
    # Tracer une ligne horizontale indiquant le niveau d'énergie
    ax[0].axhline(y=E, color=color, linestyle="--", alpha=0.6)

ax[0].set_title("Fonctions d'onde pour les 4 premiers niveaux\ndans un puits de potentiel carré infini")
ax[0].set_xlabel("Position x")
ax[0].set_ylabel("Fonction d'onde (décalée par le niveau d'énergie)")
ax[0].legend()
ax[0].grid()

# --- Graphique des densités de probabilité décalées par leur niveau d'énergie ---
ax[1].plot(x_values, potential, label="Puits de potentiel\n(côtés élevés)", color='black', linestyle="-")

for n, (prob_val, E, color) in zip(niveaux, zip(prob_vals, energies, colors)):
    prob_shifted = prob_val + E
    ax[1].plot(x_values, prob_shifted, label=r"$|\Psi_{{{}}}(x)|^2$".format(n), color=color)
    ax[1].vlines(0, ymin=0, ymax=E, linestyles='dashed', colors=color)
    ax[1].vlines(a, ymin=0, ymax=E, linestyles='dashed', colors=color)
    ax[1].fill_between(x_values, E, prob_shifted, where=((x_values >= 0) & (x_values <= a)), 
                        color=color, alpha=0.3)
    ax[1].axhline(y=E, color=color, linestyle="--", alpha=0.6)

ax[1].set_title("Densités de probabilité pour les 4 premiers niveaux\ndans un puits de potentiel carré infini")
ax[1].set_xlabel("Position x")
ax[1].legend()
ax[1].grid()

# Définir une limite en ordonnée pour mieux visualiser le potentiel
plt.ylim([0, V0 * 1.1])
plt.tight_layout()
plt.show()
